In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

## defining images's path dataframe
image_dir1 = Path('./iris-test/iris-setosa')
filepaths1 = pd.Series(list(image_dir1.glob(r'**/*.jpg')), name='Filepath').astype(str)
df_filepaths1 = pd.DataFrame(filepaths1)

image_dir2 = Path('./iris-test/iris-versicolour')
filepaths2 = pd.Series(list(image_dir2.glob(r'**/*.jpg')), name='Filepath').astype(str)
df_filepaths2 = pd.DataFrame(filepaths2)

image_dir3 = Path('./iris-test/iris-virginica')
filepaths3 = pd.Series(list(image_dir3.glob(r'**/*.jpg')), name='Filepath').astype(str)
df_filepaths3 = pd.DataFrame(filepaths3)

df_images = pd.concat([df_filepaths1, df_filepaths2, df_filepaths3], ignore_index=True)

## defining csv data dataframe
datos = pd.read_csv('./iris.csv')
df = datos["PetalLengthCm"]
df_datos = pd.DataFrame(datos)
df_datos.drop('Id', inplace=True, axis=1)
df_datos.drop('Species', inplace=True, axis=1)

## concatenate both dataframes
df_final = pd.concat([df_datos,df_images], axis=1)

print(df_final)

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0              5.1           3.5            1.4           0.2   
1              4.9           3.0            1.4           0.2   
2              4.7           3.2            1.3           0.2   
3              4.6           3.1            1.5           0.2   
4              5.0           3.6            1.4           0.2   
..             ...           ...            ...           ...   
145            6.7           3.0            5.2           2.3   
146            6.3           2.5            5.0           1.9   
147            6.5           3.0            5.2           2.0   
148            6.2           3.4            5.4           2.3   
149            5.9           3.0            5.1           1.8   

                                              Filepath  
0    iris-test\iris-setosa\iris-01ab65973fd487a6cee...  
1    iris-test\iris-setosa\iris-0797945218a97d6e525...  
2    iris-test\iris-setosa\iris-0c826b6f4648edf5

In [7]:
# X_train,X_test,Y_train,Y_test = train_test_split(df_images,df_datos, train_size=0.7)

# train_df = pd.concat([X_train,Y_train], axis=1)
# test_df = pd.concat([X_test,Y_test], axis=1)

## defining training and test dataframes
train_df, test_df = train_test_split(df_final, train_size=0.7)

#train_df = np.asarray(train_df).astype(np.float32)

print(train_df)

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
98             5.1           2.5            3.0           1.1   
134            6.1           2.6            5.6           1.4   
27             5.2           3.5            1.5           0.2   
117            7.7           3.8            6.7           2.2   
22             4.6           3.6            1.0           0.2   
..             ...           ...            ...           ...   
87             6.3           2.3            4.4           1.3   
63             6.1           2.9            4.7           1.4   
48             5.3           3.7            1.5           0.2   
57             4.9           2.4            3.3           1.0   
50             7.0           3.2            4.7           1.4   

                                              Filepath  
98   iris-test\iris-versicolour\iris-7dcc69fa294fe0...  
134  iris-test\iris-virginica\iris-697588d7bb5e1704...  
27   iris-test\iris-setosa\iris-706c80d1f831870c

In [8]:
## defining generation functions
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

## training, validation and testing parameters
columns = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col=columns,
    #y_col='PetalLengthCm',
    color_mode='rgb',
    class_mode='raw',
    batch_size=10,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col=columns,
    #y_col='PetalLengthCm',
    color_mode='rgb',
    class_mode='raw',
    batch_size=10,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col=columns,
    #y_col='PetalLengthCm',
    color_mode='rgb',
    class_mode='raw',
    batch_size=10,
    shuffle=False
)

Found 84 validated image filenames.
Found 21 validated image filenames.
Found 45 validated image filenames.


In [13]:
## training process
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(4, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse',
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/5
9/9 [==============================] - 5s 451ms/step - loss: 16.0109 - val_loss: 13.3551
Epoch 2/5
9/9 [==============================] - 3s 358ms/step - loss: 13.7122 - val_loss: 9.5461
Epoch 3/5
9/9 [==============================] - 3s 316ms/step - loss: 7.6885 - val_loss: 3.1221
Epoch 4/5
9/9 [==============================] - 3s 330ms/step - loss: 3.2838 - val_loss: 2.6912
Epoch 5/5
9/9 [==============================] - 3s 323ms/step - loss: 2.3314 - val_loss: 1.7143


In [10]:
## error metrics
predicted_parameters = np.squeeze(model.predict(test_images))
true_parameters = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_parameters, predicted_parameters)
print("Test R^2 Score: {:.5f}".format(r2))

null_rmse = np.sqrt(np.sum((true_parameters - np.mean(true_parameters))**2) / len(true_parameters))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

5/5 [==============================] - 0s 66ms/step
     Test RMSE: 1.30322
Test R^2 Score: -1.51903
Null/Baseline Model Test RMSE: 3.82270


In [29]:
## prediction with test data
model.predict(test_images)


5/5 [==============================] - 1s 91ms/step


array([[6.223924  , 3.395778  , 4.136097  , 1.0998406 ],
       [5.698388  , 3.1077702 , 3.7943544 , 1.001617  ],
       [4.3792515 , 2.391115  , 2.9161694 , 0.7804091 ],
       [8.121403  , 4.4302363 , 5.39423   , 1.4264172 ],
       [5.848544  , 3.1904497 , 3.8899388 , 1.029964  ],
       [4.5032015 , 2.4544394 , 2.992092  , 0.80701387],
       [5.4305086 , 2.9646218 , 3.6147165 , 0.95759267],
       [4.404561  , 2.4043174 , 2.9369397 , 0.77857393],
       [2.7803154 , 1.5206186 , 1.8604645 , 0.5036869 ],
       [5.9045553 , 3.218752  , 3.9213216 , 1.0501597 ],
       [4.3763046 , 2.3858657 , 2.9144943 , 0.7801597 ],
       [3.786268  , 2.063833  , 2.5174434 , 0.68165815],
       [6.1193547 , 3.3385763 , 4.0681453 , 1.0810325 ],
       [5.1915874 , 2.8333967 , 3.460328  , 0.9144415 ],
       [6.2305355 , 3.3953073 , 4.1361437 , 1.1075397 ],
       [4.645231  , 2.531512  , 3.0897243 , 0.8293212 ],
       [5.2899065 , 2.8921158 , 3.5352054 , 0.91919196],
       [3.931265  , 2.1469214 ,

In [30]:
## prediction with test data with respective image
test_prediction = pd.DataFrame(model.predict(test_images), columns=['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm'])

test_prediction.reset_index(drop=True, inplace=True)
test_df['Filepath'].reset_index(drop=True, inplace=True)

prediction_with_img = pd.concat([test_prediction, test_df['Filepath']], axis=1)

print(prediction_with_img)

5/5 [==============================] - 0s 69ms/step
    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0        6.223924      3.395778       4.136097      1.099841   
1        5.698388      3.107770       3.794354      1.001617   
2        4.379251      2.391115       2.916169      0.780409   
3        8.121403      4.430236       5.394230      1.426417   
4        5.848544      3.190450       3.889939      1.029964   
5        4.503201      2.454439       2.992092      0.807014   
6        5.430509      2.964622       3.614717      0.957593   
7        4.404561      2.404317       2.936940      0.778574   
8        2.780315      1.520619       1.860464      0.503687   
9        5.904555      3.218752       3.921322      1.050160   
10       4.376305      2.385866       2.914494      0.780160   
11       3.786268      2.063833       2.517443      0.681658   
12       6.119355      3.338576       4.068145      1.081033   
13       5.191587      2.833397       3.460328      